In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline
pd.set_option('max_colwidth',1000)

In [2]:
# Load the loan data
df = pd.read_pickle('data_cleaned.pkl')

In [3]:
# Categorical features from data-cleanup.ipynb
cat_features = ['grade', 'sub_grade', 'emp_length', 'home_ownership', 'verification_status', 
                'loan_status', 'purpose', 'addr_state', 'initial_list_status', 'application_type', 
                'disbursement_method']

for y in cat_features:
    print(y + " has " + str(len(df[y].unique())) + " unique values")
    # df = df.join(pd.get_dummies(df[y], prefix=y))
    # df.drop(y, axis=1, inplace=True)

grade has 7 unique values
sub_grade has 35 unique values
emp_length has 12 unique values
home_ownership has 3 unique values
verification_status has 3 unique values
loan_status has 2 unique values
purpose has 14 unique values
addr_state has 51 unique values
initial_list_status has 2 unique values
application_type has 2 unique values
disbursement_method has 2 unique values


In [4]:
important_cat_features = ['purpose', 'grade', 'emp_length', 'home_ownership', 'sub_grade', 'verification_status', 
                'addr_state', 'initial_list_status', 'application_type',  'disbursement_method']

# Since converting one-hot encoding can drastically increase the size of the table, we need to be selective.
# Hence, I am skipping sub_grade and addr_state
for y in important_cat_features:
    print(y)
    if len(df[y].unique()) < 15:
        df = df.join(pd.get_dummies(df[y], prefix=y))
        print("One-hot encoding done")
    df.drop(y, axis=1, inplace=True)
    print("Dropped")
    df.info()

purpose
One-hot encoding done
Dropped
<class 'pandas.core.frame.DataFrame'>
Int64Index: 2831339 entries, 0 to 421094
Columns: 114 entries, loan_amnt to purpose_wedding
dtypes: int64(90), object(10), uint8(14)
memory usage: 2.2+ GB
grade


MemoryError: 